# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Using-extreme-gradient-boosting-to-detect-glottal-closure-instants-in-speech-signal" data-toc-modified-id="Using-extreme-gradient-boosting-to-detect-glottal-closure-instants-in-speech-signal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using extreme gradient boosting to detect glottal closure instants in speech signal</a></div><div class="lev2 toc-item"><a href="#Training-and-evaluating-the-classifier-on-UWB-data" data-toc-modified-id="Training-and-evaluating-the-classifier-on-UWB-data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training and evaluating the classifier on UWB data</a></div><div class="lev2 toc-item"><a href="#CMU-data" data-toc-modified-id="CMU-data-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>CMU data</a></div>

# Using extreme gradient boosting to detect glottal closure instants in speech signal

This is an example of a Python code to train and test an extreme gradient boosting (XGB) classifier used to detect glottal closure instants (GCIs) in the speech signal. See the [corresponding paper](paper/matousek_ICASSP2019_paper.pdf) for more details.

[Scikit-learn](http://scikit-learn.org/stable/) and [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) toolkits are used to train and evaluate the classifier.

Prerequisities:
- [Python](https://www.python.org/) (version 3.7.0 used in this example)
- [Numpy](http://www.numpy.org/) (1.15.2)
- [Scipy](https://www.scipy.org/) (1.1.0)
- [Scikit-learn](http://scikit-learn.org/stable/) (0.20.0)
- [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) (0.80)
- [Pandas](http://pandas.pydata.org/) (0.23.4)

## Training and evaluating the classifier on UWB data

Firstly, we define the XGB classifier with the default hyper-parameters

In [2]:
from xgboost import XGBClassifier

# Init the XGB classifier
xgb = XGBClassifier(random_state=8)

The default hyper-parameters can be listed

In [34]:
xgb.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 8,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 1}

Then, the UWB data and its targets (denoted as _development dataset_ in the  [corresponding paper](paper/matousek_ICASSP2019_paper.pdf)) stored in a CSV is loaded as a Pandas dataframe. Hand-crafted GCIs were used as targets and are stored in the very first column of the dataframe. Other columns represent features; the features are those selected automatically by recursive feature elimination with extremely randomized trees used as an external estimator for feature importances (denoted as RFE-ERT in the [paper](paper/matousek_ICASSP2019_paper.pdf)).

In [36]:
import pandas as pd
# Read development data and targets into a Pandas dataframe
df = pd.read_csv('data/uwb/data.csv', sep=';')

This is an example of the dataframe. Columns are the features (except the first one which is the target) and rows correspond to negative peaks (candidates for GCI placement). The target value 0 means the peak is not a true GCI, while the value 1 represents a true GCI.

In [37]:
# Show first examples
df.head()

,target,negAmp0,negAmp+1,negAmp-1,negAmp+2,negAmp-2,negAmp+3,negAmp-3,posAmp+1,posAmp-1,...,negPeakRatio+3,negPeakRatio-3,zcr,energy,hnr,specCentroid,specRollOff,mfcc0,mfcc1,f0
0,0,0.000509,0.000882,0.000000,0.000305,0.000000,0.000916,0.000000,0.000848,0.000000,...,1.800000,0.000000,54,6.663986,0.018084,3554.503394,6875.0,7.861533,-27.152266,0.0
1,0,0.000882,0.000305,0.000509,0.000916,0.000000,0.000576,0.000000,0.000136,0.000848,...,0.653846,0.000000,48,6.610703,2.137606,3445.156903,6875.0,7.712359,-24.323951,0.0
2,0,0.000305,0.000916,0.000882,0.000576,0.000509,0.000068,0.000000,0.000102,0.000136,...,0.222222,0.000000,60,6.547511,0.000000,3515.395209,6375.0,7.319554,-27.431489,0.0
3,0,0.000916,0.000576,0.000305,0.000068,0.000882,0.000170,0.000509,0.000882,0.000102,...,0.185185,0.555556,40,6.450985,0.000000,3256.522404,6812.5,5.339055,-30.872795,0.0
4,0,0.000576,0.000068,0.000916,0.000170,0.000305,0.001322,0.000882,0.000237,0.000882,...,2.294118,1.529412,51,6.426121,0.000000,3509.230895,6250.0,6.552707,-29.550572,0.0


For the further processing, the targets and features are stored in NumPy arrays

In [30]:
import numpy as np
# 0th column is the target
y = np.array(df.values[:,0], dtype='int')
# Other columns represent the features
X = df.values[:,1:]
print('# examples: {}'.format(X.shape[0]))
print('# features: {}'.format(X.shape[1]))

# examples: 73205
# features: 37


The performance of the proposed baseline XGB classifier with the default hyper-parameters can be evaluated by [_k_-fold cross-validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html). To enable faster training, we use just 3-folds in this example; of course, more folds and/or [repetitions](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html#sklearn.model_selection.RepeatedStratifiedKFold) can be used to get more unbiased results.

In [19]:
from sklearn.model_selection import cross_val_score
# Cross-validate
scores = cross_val_score(xgb, X, y, cv=3, scoring='f1')
print('Scores: {}'.format(scores))
print('Mean CV score: {:5.2%} +/- {:5.2%}'.format(scores.mean(), scores.std()))

Scores: [0.9698307  0.97124771 0.97682144]
Mean CV score: 97.26% +/- 0.30%


Better results are expected when the hyper-parameters of the XGB model are tuned. In this simplified example, we tune just 2 hyper-parameters, the number of decision trees (*n_estimators*) and maximum depth of a tree (*max_depth*).

A simplified grid of possible hyper-parameter values is defined

In [18]:
# Define parameter grid
grid = {'n_estimators': [50, 100, 150],
        'max_depth': [1, 3, 5]}

A [grid search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) is then defined to search for the best classifier (hyper)parameters using a 3-fold cross-validation scheme ($cv=3$). F1 score is used in this example to evaluate the predicitions of each hyper-parameter setting on the validation set.

In [19]:
from sklearn.model_selection import GridSearchCV
# Define the grid search and cross-validation
gs = GridSearchCV(xgb, grid, scoring='f1', cv=3, return_train_score=False)

The grid search object is then trained.

In [20]:
# Train the grid 
gs.fit(X, y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=8,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [50, 100, 150], 'max_depth': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='f1', verbose=0)

The best parameters found by the grid search are

In [21]:
print('Best hyper-parameters found: ', gs.best_params_)

Best hyper-parameters found:  {'max_depth': 5, 'n_estimators': 100}


Mean (F1) score for the best hyper-parameters:

In [25]:
print('Mean CV score for the best hyper-paramater setting: {:5.2%}'.format(gs.best_score_))

Mean CV score for the best hyper-paramater setting: 97.48%


Summary of results for each hyper-parameter setting and each cross-validation fold could be stored to a CSV file

In [23]:
from pandas import DataFrame
# Import cross-validation results
res = DataFrame(gs.cv_results_)
# Save to CSV file
res.to_csv('cv_results.csv', sep=';')

Note that much more extensive grid search on more relevant hyper-parameters was conducted in the [corresponding paper](paper/matousek_ICASSP2019_paper.pdf). The resulting XGB model (refit on all development data) can be loaded

In [26]:
import pickle
with open('data/uwb/xgb_final.p', 'rb') as fr:
    xgb_final = pickle.load(fr)

The tuned hyper-parameters can be listed

In [41]:
xgb_final.get_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.6,
 'colsample_bytree': 0.65,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 7,
 'min_child_weight': 1,
 'missing': nan,
 'n_estimators': 1150,
 'n_jobs': 1,
 'nthread': None,
 'objective': 'binary:logistic',
 'random_state': 8,
 'reg_alpha': 1e-08,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': None,
 'silent': True,
 'subsample': 0.9}

## CMU data

The trained and tuned classifier can be evaluated on the [CMU](http://festvox.org/dbs/index.html) test datasets. Since hand-crafted GCIs are not available for these datasets, we used the [Multi-Phase Algorithm](http://www.sciencedirect.com/science/article/pii/S0167639311000094) (MPA) to detect GCIs from the contemporaneous electroglottograph (EGG) signal and used the detected GCIs as the reference ones. The reference GCIs are available in the [wavesurfer](http://www.speech.kth.se/wavesurfer) format

```
0.234687 0.234687 V
0.242312 0.242312 V
0.250250 0.250250 V
0.258062 0.258062 V
0.265937 0.265937 V
```

The most important is the first column which denotes the position of a GCI in seconds. Other columns can be ignored.

The features are stored in CSV format in an utterance-by-utterance manner. The first column now expresses the sample of a negative peak in the 16-kHz version of the speech signal, so a true GCI expressed in the wavesurfer format can be easily mapped to the corresponding row of the feature dataframe. For instance, the GCI at 0.234687 sec corresponds to the 3755th speech sample ($0.234687*16000 \doteq 3755$). The other columns represent the same RFE-ERT features as explained above.

Rows that do not correspond to any wavesurfer-based time instance do not represent a true GCI.

In [40]:
import pandas as pd
# Read an example feature set (BDL ```arctic_a0001``` utterance)
df_bdl1 = pd.read_csv('data/cmu/bdl/bdl_arctic_a0001.csv', sep=';')
df_bdl1.head()

,negPeakIdx,negAmp0,negAmp+1,negAmp-1,negAmp+2,negAmp-2,negAmp+3,negAmp-3,posAmp+1,posAmp-1,...,negPeakRatio+3,negPeakRatio-3,zcr,energy,hnr,specCentroid,specRollOff,mfcc0,mfcc1,f0
0,170,0.002408,0.000848,0.000000,0.000237,0.000000,0.000509,0.000000,0.005459,0.000000,...,0.211268,0.000000,12,5.819205,9.531323,1921.265120,5093.75,6.635018,-19.232227,0.0
1,714,0.000848,0.000237,0.002408,0.000509,0.000000,0.000949,0.000000,0.000475,0.005459,...,1.120000,0.000000,30,5.911118,9.064202,2345.460398,5812.50,4.551766,-27.449987,0.0
2,830,0.000237,0.000509,0.000848,0.000949,0.002408,0.002102,0.000000,0.003052,0.000475,...,8.857143,0.000000,42,6.291295,4.616575,2835.998739,6218.75,6.742996,-25.020588,0.0
3,988,0.000509,0.000949,0.000237,0.002102,0.000848,0.000203,0.002408,0.001424,0.003052,...,0.400000,4.733333,12,5.730694,8.145357,2038.857806,5718.75,6.141694,-20.078818,0.0
4,1109,0.000949,0.002102,0.000509,0.000203,0.000237,0.000916,0.000848,0.002577,0.001424,...,0.964286,0.892857,16,5.851574,9.596124,2571.626569,6156.25,7.016690,-20.557482,0.0


The reference GCIs and datasets for all utterances and voices can be downloaded from the following table (or directly from the ```data/cmu``` folder)

| Voice | GCIs                                 | Dataset                           |
| ----- | -------------------------------------| --------------------------------- | 
| BDL   | [x](data/cmu/bdl/bdl_ref_gci.tar.gz) | [x](data/cmu/bdl/bdl_data.tar.gz) |
| SLT   | [x](data/cmu/slt/slt_ref_gci.tar.gz) | [x](data/cmu/slt/slt_data.tar.gz) |
| KED   | [x](data/cmu/ked/ked_ref_gci.tar.gz) | [x](data/cmu/ked/ked_data.tar.gz) |

GCIs detected by different methods are stored in the ``data/cmu/<voice>`` folder where ``<voice>`` is one of the voices we experimented with: ``bdl``, ``slt``, ``ked``.

The name of the compressed file with GCIs is as follow:

``<voice>_<method>_>type>_gci``
* ``<voice>``  ...  a voice (``bdl``, ``slt``, ``ked``)
* ``<method>`` ... GCI detection method (``dypsa``, ``mmf``, ``reaper``, ``sedreams``, ``xgb``)
* ``<type>``   ... GCI type (original vs. postprocessed)
  * ``orig`` ... original GCIs as detected by each method
  * ``post`` ... postprocessed GCIs (V/U filtering, syncing with neighboring minimum negative sample)